# create and save a `UnitsDf.csv`

In [1]:
import sys, os
from pathlib import Path
import numpy as np
import pandas as pd

import readSGLX as glx
import pyks_tools as pkt

import pynapple as nap

In [2]:
# path definitions
# maybe it's possible to abbreviate this
exp_folder = Path('/media/georg/htcondor/shared-paton/georg/DAtime/data/2023-02-17_JJP-05313-dh_B_1-2-3/')
run_folder = exp_folder / 'stim_run_2_g0'
imec_bin_path = run_folder / 'stim_run_2_g0_t0.imec0.ap.bin'
ni_bin_path = run_folder / 'stim_run_2_g0_t0.nidq.bin'
ks_folder = run_folder / 'pyks2_output'
results_folder = ks_folder / 'results'

In [10]:
# path definitions
exp_folder = Path('/media/georg/htcondor/shared-paton/georg/DAtime/data/batch_24a/2024-06-06_JJP-08672_dh_1-6-1')
run_folder = exp_folder / 'stim_run_1_g0'
imec_bin_path = run_folder / 'stim_run_1_g0_t0.imec0.ap.bin'
ni_bin_path = run_folder / 'stim_run_1_g0_t0.nidq.bin'
ks_folder = run_folder / 'ibl_sorter_results'
results_folder = ks_folder / 'results'
os.makedirs(results_folder, exist_ok=True)

In [2]:
exp_folder = Path('/media/georg/htcondor/shared-paton/georg/DAtime/data/batch_24a_sorted/2024-06-08_JJP-08628_dh_5-6-1/')
run_folder = exp_folder / 'stim_run_3_g0'
imec_bin_path = run_folder / 'stim_run_3_g0_t0.imec0.ap.bin'
ni_bin_path = run_folder / 'stim_run_3_g0_t0.nidq.bin'
ks_folder = run_folder / 'ibl_sorter_results'
results_folder = ks_folder / 'results'

In [3]:
UnitsDf = pkt.load_pyks2_result(ks_folder, imec_bin_path)
UnitsDf

,cluster_id,ContamPct,Amplitude,group,good,n_spikes,frate
0,0,9.1,83.6,good,True,24952,5.337907
1,1,16.1,78.2,mua,True,26259,5.617509
2,2,2.0,212.2,good,True,47955,10.258869
3,3,4.7,91.0,good,True,90316,19.321031
4,4,14.8,89.4,mua,True,9524,2.037441
...,...,...,...,...,...,...,...
589,589,27.9,90.1,mua,True,864,0.184833
590,590,13.8,80.1,mua,True,1978,0.423148
591,591,0.0,108.7,good,True,365,0.078083
592,592,0.0,119.8,good,True,847,0.181196


In [6]:
# store it
# because it depends on the sorting
os.makedirs(results_folder, exist_ok=True)
UnitsDf.to_csv(results_folder / 'UnitsDf.csv', index=None)

In [7]:
# loading data for pynappling
spike_times, spike_templates = pkt.load_spikes(ks_folder, imec_bin_path)
unit_ids = UnitsDf['cluster_id']

units_dict = pkt.reformat_spikes(spike_times, spike_templates, unit_ids)
units_Ts = {}
for unit_id, spike_times in units_dict.items():
    units_Ts[unit_id] = nap.Ts(t=spike_times)

units = nap.TsGroup(units_Ts)
units.set_info(UnitsDf)

In [8]:
# store as a pynapple object
units.save(str(results_folder / 'units.npz'))